In [1]:
import nltk
nltk.download('punkt')

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model

[nltk_data] Downloading package punkt to C:\Users\ADITYA
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
df=pd.read_csv(r"C:\Users\ADITYA SINGH\Downloads\NLP_train.csv")
print(df.head())

                 id                                       comment_text  toxic
0  e617e2489abe9bca  "\r\n\r\n A barnstar for you! \r\n\r\n  The De...      0
1  9250cf637294e09d  "\r\n\r\nThis seems unbalanced.  whatever I ha...      0
2  ce1aa4592d5240ca  Marya Dzmitruk was born in Minsk, Belarus in M...      0
3  48105766ff7f075b      "\r\n\r\nTalkback\r\n\r\n Dear Celestia...  "      0
4  0543d4f82e5470b6  New Categories \r\n\r\nI honestly think that w...      0


In [5]:
df.drop(columns = ['id'], inplace=True)
df.head()

,comment_text,toxic
0,"""\r\n\r\n A barnstar for you! \r\n\r\n The De...",0
1,"""\r\n\r\nThis seems unbalanced. whatever I ha...",0
2,"Marya Dzmitruk was born in Minsk, Belarus in M...",0
3,"""\r\n\r\nTalkback\r\n\r\n Dear Celestia... """,0
4,New Categories \r\n\r\nI honestly think that w...,0


In [7]:
df.comment_text[2]

'Marya Dzmitruk was born in Minsk, Belarus in March 19, 1992. Her mother, Olga Nikolaevna Moroz was born in Baranovichi, Belarus and her father was born in Brest, Belarus. She is second child in the family. Her parents divorced in 1998 and soon after her father remarried and had two more children. \r\nMarya, at the age of 4 began doing gymnastics, but quit two years later because she was denied a medal in a competition, where her age was incorrectly marked. When she turned 6 years old, she got admitted to Music School #4 in Minsk, class of violin, and to Public School #66 with piano classes as a main course. At the age of 11, Marya starred in Belarusfilm movie called “Dunechka”. Soon after she started to play in theatre and was featured in television shows. By 2005 her mother decided to move to United States. In September of 2005 Marya went to her first American school, Ingrid B. Lacy Middle School. She graduated in Spring 2006 and traveled back to Belarus for 2 months. In August 2006 

In [8]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
    

[nltk_data] Downloading package stopwords to C:\Users\ADITYA
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
            result.append(token)
            
    return result

In [10]:
df['clean'] = df['comment_text'].apply(preprocess)

In [11]:
df.clean[0]

['barnstar',
 'defender',
 'wiki',
 'barnstar',
 'like',
 'edit',
 'kayastha',
 'page',
 'lets',
 'form',
 'solidarity',
 'group',
 'malign',
 'article',
 'subject',
 'matter',
 'propose',
 'folloing',
 'group',
 'united',
 'intellectuals',
 'kayastha',
 'ethinicty',
 'racist',
 'castist',
 'abuse',
 'uifkearca']

In [12]:
print(df.clean[0],df.comment_text[0])

['barnstar', 'defender', 'wiki', 'barnstar', 'like', 'edit', 'kayastha', 'page', 'lets', 'form', 'solidarity', 'group', 'malign', 'article', 'subject', 'matter', 'propose', 'folloing', 'group', 'united', 'intellectuals', 'kayastha', 'ethinicty', 'racist', 'castist', 'abuse', 'uifkearca'] "

 A barnstar for you! 

  The Defender of the Wiki Barnstar I like your edit on the Kayastha page. Lets form a solidarity group against those who malign the article and its subject matter. I propose the folloing name for the group.

United intellectuals' front of Kayastha ethinicty against racist or castist abuse (UIFKEARCA)   "


In [17]:
list_of_words = []
for i in df.clean:
    for j in i:
        list_of_words.append(j)
total_words = (list(set(list_of_words)))
print(len(total_words))

20047


In [14]:
print(len(list_of_words))

136400


In [16]:
maxlen=0
for i in df.clean:
    maxlen=max(maxlen,len(i))
print(maxlen)

712


In [21]:
counter={}
c=0
for i in total_words:
    c=0
    for j in df.clean:
        for k in j:
            if i==k:
                c=c+1
    counter[i]=c

In [22]:
print(counter)

{'subordinate': 1, 'summon': 1, 'joker': 2, 'understate': 1, 'wrestling': 3, 'honorific': 1, 'turbulent': 1, 'waters': 1, 'regard': 12, 'scan': 3, 'theorists': 2, 'provides': 19, 'beings': 5, 'future': 68, 'sermises': 1, 'consequence': 9, 'flensing': 1, 'diggins': 1, 'accusers': 1, 'kinds': 10, 'beginners': 1, 'flees': 1, 'ibbur': 1, 'professionally': 1, 'skerdilaidas': 2, 'planar': 1, 'austen': 1, 'moondyne': 1, 'warm': 2, 'zeno': 2, 'linger': 1, 'excess': 1, 'ghosts': 1, 'tweeted': 1, 'remand': 2, 'seped': 1, 'wear': 9, 'austin': 4, 'zohar': 2, 'garage': 1, 'rivalry': 2, 'exposure': 5, 'coupon': 1, 'thtp': 2, 'iterate': 1, 'cerberus': 1, 'caption': 10, 'culvert': 1, 'effort': 25, 'diseases': 1, 'upstater': 2, 'stealth': 1, 'propsing': 1, 'inoculation': 1, 'ryulong': 2, 'mindedness': 1, 'aristrocats': 1, 'nicaraguans': 1, 'replaced': 14, 'meen': 1, 'formulated': 2, 'peisers': 1, 'mars': 4, 'fits': 11, 'frustratingly': 1, 'speedclimb': 1, 'usages': 2, 'adjust': 1, 'blaming': 2, 'sadie'

In [23]:
df['clean_joined'] = df['clean'].apply(lambda x: " ".join(x))

In [24]:
from nltk import word_tokenize
tokenizer = Tokenizer(num_words = len(total_words))
tokenizer.fit_on_texts(df.clean_joined)

In [28]:
df['tokenized']= tokenizer.texts_to_sequences(df.clean_joined)

In [48]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.tokenized, df.toxic, test_size = 0.3)

In [49]:
print(df.tokenized[0])

[613, 6604, 34, 613, 6, 9, 6605, 2, 1528, 231, 9653, 189, 9654, 1, 47, 153, 996, 9655, 189, 258, 9656, 6605, 9657, 705, 9658, 517, 9659]


In [50]:
print(df.clean_joined[0])

barnstar defender wiki barnstar like edit kayastha page lets form solidarity group malign article subject matter propose folloing group united intellectuals kayastha ethinicty racist castist abuse uifkearca


In [51]:
df.head()


,comment_text,toxic,clean,clean_joined,tokenized
0,"""\r\n\r\n A barnstar for you! \r\n\r\n The De...",0,"[barnstar, defender, wiki, barnstar, like, edi...",barnstar defender wiki barnstar like edit kaya...,"[613, 6604, 34, 613, 6, 9, 6605, 2, 1528, 231,..."
1,"""\r\n\r\nThis seems unbalanced. whatever I ha...",0,"[unbalanced, said, mathsci, said, extreme, unp...",unbalanced said mathsci said extreme unpleasan...,"[3626, 60, 5166, 60, 1529, 5167, 72, 199, 1530..."
2,"Marya Dzmitruk was born in Minsk, Belarus in M...",0,"[marya, dzmitruk, born, minsk, belarus, march,...",marya dzmitruk born minsk belarus march mother...,"[2556, 6608, 523, 6609, 3172, 647, 347, 5171, ..."
3,"""\r\n\r\nTalkback\r\n\r\n Dear Celestia... """,0,"[talkback, dear, celestia]",talkback dear celestia,"[4257, 585, 9680]"
4,New Categories \r\n\r\nI honestly think that w...,0,"[categories, honestly, think, need, categories...",categories honestly think need categories char...,"[1078, 1532, 7, 28, 1078, 706, 1078, 6, 706, 4..."


In [ ]:
padded_train = pad_sequences(x_train,maxlen = 40, padding = 'post', truncating = 'post')
padded_train= padded_train.reshape(3500,40)
padded_test = pad_sequences(x_test,maxlen = 40, truncating = 'post')
padded_test = padded_test.reshape(1500,40)

In [71]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train1 = sc.fit_transform(padded_train)
x_test1 = sc.transform(padded_test)

0.9106666666666666

In [76]:
y_pred = classifier.predict(x_test1)

In [77]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1366,    0],
       [ 134,    0]], dtype=int64)

In [82]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
acc=accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred)
rec=recall_score(y_test,y_pred)
print(acc,f1,rec)

0.9106666666666666 0.0 0.0


# next part as model was only focussing on 0's